In [ ]:
pip install requests beautifulsoup4


In [ ]:
import requests
from bs4 import BeautifulSoup
import os

# URL base do site
base_url = "https://acordaos.economia.gov.br"

# URL inicial da pesquisa
search_url = "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=RENDIMENTOS+TRIBUTÁVEIS+e+ALUGUÉIS&fq=ano_sessao_s:%222024%22"

# Iniciar uma sessão de requests
session = requests.Session()

# Desabilitar avisos de certificado SSL
requests.packages.urllib3.disable_warnings()

# Função para baixar o PDF
def download_pdf(pdf_url):
    # Extrair o nome do arquivo a partir da URL
    pdf_name = pdf_url.split('/')[-1]
    response = session.get(pdf_url, verify=False)
    if response.status_code == 200:
        # Salvar o conteúdo do PDF em um arquivo local
        with open(pdf_name, 'wb') as f:
            f.write(response.content)
        print(f"PDF baixado: {pdf_name}")
    else:
        print(f"Falha ao baixar o PDF: {pdf_url}")

# Função para fazer o scraping de uma página
def scrape_page(url):
    response = session.get(url, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar todos os links de PDFs na página
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.endswith('.pdf'):
            if href.startswith('http'):
                pdf_url = href
            else:
                pdf_url = base_url + href
            download_pdf(pdf_url)

    # Procurar o link para a próxima página
    next_page = soup.find('a', class_='next-page')
    if next_page:
        href = next_page['href']
        if href.startswith('http'):
            next_url = href
        else:
            next_url = base_url + href
        return next_url
    else:
        return None

# URL atual começa com a URL de pesquisa
current_url = search_url

# Loop para percorrer todas as páginas
while current_url:
    print(f"Acessando página: {current_url}")
    current_url = scrape_page(current_url)


Acessando página: https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=RENDIMENTOS+TRIBUTÁVEIS+e+ALUGUÉIS&fq=ano_sessao_s:%222024%22
PDF baixado: 11080014681200822_7020812.pdf
PDF baixado: 12448733329201141_7065778.pdf
PDF baixado: 10983720541201075_7047598.pdf
PDF baixado: 18239008049200848_7043159.pdf
PDF baixado: 10830723787201123_7142651.pdf
PDF baixado: 13888004030201089_7130967.pdf
PDF baixado: 10166006862201097_7043119.pdf
PDF baixado: 10930723352201231_7020816.pdf
PDF baixado: 19555724146202139_7044916.pdf
PDF baixado: 15463000073200978_7048658.pdf
Acessando página: https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&&fq=ano_sessao_s%3A%222024%22&q=RENDIMENTOS+TRIBUT%C3%81VEIS+e+ALUGU%C3%89IS&start=10
PDF baixado: 10930723352201231_7020816.pdf
PDF baixado: 19555724146202139_7044916.pdf
PDF baixado: 13887720002202094_7092736.pdf
PDF baixado: 10283722827201911_7071178.pdf
PDF baixado: 10166009780200961_7043120.pdf
PDF baixado: 1232

In [ ]:
import zipfile
import os
from google.colab import files

# Nome do arquivo ZIP que será criado
nome_zip = 'acordaos_pdfs.zip'

# Caminho para a pasta raiz '/content/'
pasta_raiz = '/content/'

# Criar um arquivo ZIP contendo todos os PDFs em '/content/'
with zipfile.ZipFile(nome_zip, 'w') as zipf:
    for file in os.listdir(pasta_raiz):
        if file.endswith('.pdf'):
            caminho_arquivo = os.path.join(pasta_raiz, file)
            # Adicionar o arquivo ao ZIP
            zipf.write(caminho_arquivo, arcname=file)
print(f"Arquivo ZIP '{nome_zip}' criado com sucesso.")

# Fazer o download do arquivo ZIP para o seu computador
files.download(nome_zip)


Arquivo ZIP 'acordaos_pdfs.zip' criado com sucesso.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os

# ... other code

# Get the number of files in the directory
num_files = len(os.listdir(pasta_raiz))

# Print the number of files
print(f"Number of files in '{pasta_raiz}': {num_files}")

Number of files in '/content/': 361


In [ ]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 31.8 MB/s eta 0:00:00


In [ ]:
import os
import fitz  # PyMuPDF

def converter_pdf_para_texto(pdf_file):
    try:
        doc = fitz.open(pdf_file)
        texto = ""
        for pagina in doc:
            texto += pagina.get_text("text")
        doc.close()
        return texto
    except Exception as e:
        print(f"Erro ao converter PDF {pdf_file}: {e}")
        return None

def processar_pasta_pdfs(pasta_pdf, txt_saida_final):
    with open(txt_saida_final, 'w', encoding='utf-8') as arquivo_saida:
        for pdf_nome in os.listdir(pasta_pdf):
            if pdf_nome.endswith('.pdf'):
                prefixo = pdf_nome.split('_')[0]
                caminho_pdf = os.path.join(pasta_pdf, pdf_nome)

                texto_pdf = converter_pdf_para_texto(caminho_pdf)
                if texto_pdf:
                    arquivo_saida.write(f"\n=== Início do documento {prefixo} ===\n")
                    arquivo_saida.write(texto_pdf)
                    arquivo_saida.write(f"\n=== Fim do documento {prefixo} ===\n")
                    print(f"Processado: {pdf_nome}")

# Exemplo de uso
pasta_pdf = '/content/drive/MyDrive/Scrapping_CARF/acordaos_pdfs'  # Substitua pelo caminho da pasta com os PDFs
txt_saida_final = 'arquivo_saida_final.txt'  # Nome do arquivo de saída de texto

# Processar todos os PDFs da pasta
processar_pasta_pdfs(pasta_pdf, txt_saida_final)


Processado: 11080014681200822_7020812.pdf
Processado: 12448733329201141_7065778.pdf
Processado: 18239008049200848_7043159.pdf
Processado: 10983720541201075_7047598.pdf
Processado: 10830723787201123_7142651.pdf
Processado: 13888004030201089_7130967.pdf
Processado: 10166006862201097_7043119.pdf
Processado: 15463000073200978_7048658.pdf
Processado: 10930723352201231_7020816.pdf
Processado: 19555724146202139_7044916.pdf
Processado: 13887720002202094_7092736.pdf
Processado: 10283722827201911_7071178.pdf
Processado: 10166009780200961_7043120.pdf
Processado: 13027720002201808_7092735.pdf
Processado: 12326001679200915_7022418.pdf
Processado: 13896721319201903_7059217.pdf
Processado: 15463002001201007_7130962.pdf
Processado: 10166001933200921_7114700.pdf
Processado: 19647011603200825_7027448.pdf
Processado: 11080725682201011_7130927.pdf
Processado: 13887720193201951_7092737.pdf
Processado: 10865720853201378_7051795.pdf
Processado: 10840722084201240_7047602.pdf
Processado: 18239001639200921_7130